In [30]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import AnimalShelter class
from animalShelter import AnimalShelter

# Read the logo image and encode it
with open('Grazioso Salvare Logo.png', 'rb') as f:
    logo_image = f.read()
encoded_logo_image = base64.b64encode(logo_image).decode('utf-8')

# Define the username, password, and shelter
username = "aacuser"
password = "Turtle1"
shelter = AnimalShelter()

default_lat = 30.75
default_long = -97.48

df = pd.DataFrame.from_records(shelter.read({}))
df.dropna(subset=['breed', 'age_upon_outcome_in_weeks'], inplace=True)
# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)
print(df)
## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([

    html.Center(html.B(html.H1('Grazioso Salvare Database'))),
    html.Center(html.B(html.H1('By Wilfredo Mendez'))),
    html.Hr(),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={'width': '200px', 'height': '200px'})),
    html.Center(html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        ))),
    #html.Div(className = 'buttonRow',
    #         style = {'display': 'flex'},
     #            children = [
      #               html.Button(id = 'submit-button-one', n_clicks = 0, children = "Cats"),
       #              html.Button(id = 'submit-button-two', n_clicks = 0, children = "Dogs")
        #         ]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable="single",
                         row_selectable= "single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows= [0],
                         page_action="native",
                         page_current=0,
                         page_size=10
                        ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
# code to filter interactive data table with MongoDB queries
def update_dashboard(filter_type):
    # Initialize an empty DataFrame
    df = pd.DataFrame(columns=['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color', 
                               'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype', 
                               'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long', 
                               'age_upon_outcome_in_weeks', 'age', 'species'])

    if filter_type == 'mount':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                              "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))
    elif filter_type == 'water':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))
    elif filter_type == 'disaster':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                              "Bloodhound","Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
        }))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))

    # Drop unnecessary columns
    df.drop(columns=['_id'], inplace=True)
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')

    return data, columns

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    #if viewData is None:
        #return []  # Return an empty list if viewData is None
    
    # Convert viewData to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Check if the DataFrame is empty or does not contain the 'breed' column
    if dff.empty or 'breed' not in dff.columns:
        return []  # Return an empty list if the DataFrame is empty or does not contain 'breed'

    pie_chart = dcc.Graph(
        id='graph_id',
        figure=px.pie(
            dff, 
            values='age_upon_outcome_in_weeks', 
            names='breed',
            title='Breeds',
            labels={'breed': 'Breed'},  # Set the label for the 'breed' column
            hover_data=['age_upon_outcome_in_weeks'],  # Add hover data for age
            width=600,  # Set the width of the chart
            height=600,  # Set the height of the chart
            )
    )

    return [pie_chart]
        

    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None or index is None:
        return []  # Return an empty list if data or index is None
    
    # Convert viewData to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Extract the selected row index
    row_index = index[0] if index else None
    
    # Check if the row index is valid
    if row_index is not None and row_index >= 0 and row_index < len(dff):
        # Extract latitude and longitude from the selected row
        lat = dff.iloc[row_index]['location_lat']
        lon = dff.iloc[row_index]['location_long']
        
        # Create a marker for the selected animal
        marker = dl.Marker(position=[lat, lon], children=[
            dl.Tooltip(dff.iloc[row_index]['name']),
            dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[row_index]['name'])
            ])
        ])
        
        # Create the map with the marker
        map_children = [
            dl.Map(
                style={'width': '1000px', 'height': '500px'},
                center=[lat, lon],
                zoom=9,
                children=[
                    dl.TileLayer(id="base-layer-id"),
                    marker  # Add the marker to the map
                ]
            )
        ]
    else:
        # If the selected row index is invalid, display an empty map
        map_children = [
            dl.Map(
                style={'width': '700px', 'height': '500px'},
                center=[default_lat, default_long],  # Use default center coordinates
                zoom=9,
                children=[
                    dl.TileLayer(id="base-layer-id")
                ]
            )
        ]

    return map_children

                

app.run_server(debug=True)

      rec_num age_upon_outcome animal_id animal_type  \
0         1.0          3 years   A746874         Cat   
1         2.0           1 year   A725717         Cat   
2         5.0          2 years   A691584         Dog   
3         3.0          2 years   A716330         Dog   
4        12.0           1 year   A664843         Dog   
...       ...              ...       ...         ...   
9942   9980.0          2 years   A670771         Dog   
9943   9997.0          5 years   A745429         Cat   
9944   9998.0         2 months   A697165         Dog   
9945   9999.0         2 months   A665304         Dog   
9946  10000.0          4 weeks   A728994       Other   

                                         breed         color date_of_birth  \
0                       Domestic Shorthair Mix   Black/White    2014-04-10   
1                       Domestic Shorthair Mix  Silver Tabby    2015-05-02   
2                       Labrador Retriever Mix     Tan/White    2012-11-06   
3              